In [1]:
import numpy as np

modelnames = ['Layered Connectivity (SCB)',
              'Layered Connectivity (XT3D)',
              'Full Connectivity (SCB)',
              'Full Connectivity (XT3D)']

fig_titles = ['a) Layered Connectivity Grid - Standard Conductance Based Formulation',
               'b) Layered Connectivity Grid - XT3D Formulation',
               'c) Full Connectivity Grid - Standard Conductance Based Formulation',
               'd) Full Connectivity Grid - XT3D Formulation']

In [2]:
def fig2(i, vmin, vmax, extent, scale_chan, scale_bndy, scale_dom, normalize):    
    
    fig2cell(fig, i, vmin, vmax, extent, scale_chan, scale_dom, normalize)
    fig2face(fig, i, vmin, vmax, extent, scale_chan, scale_bndy, scale_dom, normalize)
    fig2head(fig, i, vmin, vmax, extent, scale_chan)

In [3]:
def fig2cell(fig, i, vmin, vmax, extent, scale_chan, scale_dom, normalize):    

    qx_chan = np.ma.masked_where(icelltype != 1, qx)
    qy_chan = np.ma.masked_where(icelltype != 1, qy)
    qz_chan = np.ma.masked_where(icelltype != 1, qz)
    qx_dom = np.ma.masked_where(icelltype != 0, qx)
    qy_dom = np.ma.masked_where(icelltype != 0, qy)
    qz_dom = np.ma.masked_where(icelltype != 0, qz)
    j = i * 3 + 1
    ax = plt.subplot(4,3,j,aspect="equal")
    xs = flopy.plot.PlotCrossSection(model=gwf, line={"line": [(0, 0.5),(Lx, 0.5)]}, extent=extent,)
    cbct = xs.plot_array(a = icelltype, cmap = 'gray', alpha=0.4, vmin = -4, vmax = 1)
    xs.plot_vector(qx_chan, qy_chan, qz_chan, scale=scale_chan, normalize=normalize, color="black")
    xs.plot_vector(qx_dom, qy_dom, qz_dom, scale=scale_dom, normalize=normalize, color="gray")
    xs.plot_grid(lw = 0.1, color = 'black')
    if j < 10: ax.set_xticks([])
    if j == 11: ax.set_xlabel('x (m)', size = 10)
    ax.set_ylabel('z (m)', size = 10) 
    ax.set_title(fig_titles[i], size = 10, loc = 'left')
    ax.annotate("Flux magnitude = %0.2f m/d\nFlux direction = %0.2f$^\circ$\nFlow error = %0.1f %s" 
               %(q_all[i][0], q_all[i][1], q_all[i][2]*100, '%'),
                (0.5,12.1), va='top', size = 8, bbox=dict(facecolor = 'white'))

In [4]:
def fig2face(fig, i, vmin, vmax, extent, scale_chan, scale_bndy, scale_dom, normalize):
        
    gp = d2d.get_gridprops_disu6()
    nja = gp["nja"]
    ihc = gp["ihc"]

    qxface = np.zeros((nja))
    qzface = np.zeros((nja))
    for iconn in range(nja):
        if ihc[iconn] > 0:
            qxface[iconn] = qnface[iconn]
        else:
            qzface[iconn] = qnface[iconn]
    j = i * 3 + 2
    ax = plt.subplot(4,3,j,aspect="equal")
    xs = flopy.plot.PlotCrossSection(model=gwf, line={"line": [(0, 0.5),(Lx, 0.5)]}, extent=extent,)
    cbct = xs.plot_array(a = icelltype, cmap = 'gray', alpha=0.4, vmin = -4, vmax = 1)
    qxplot = np.ma.masked_where(ifacetype != 0, qxface)
    qzplot = np.ma.masked_where(ifacetype != 0, qzface)
    plt.quiver(xface, zface, qxplot, qzplot, scale=scale_dom, pivot='mid', color='blue')
    qxplot = np.ma.masked_where(ifacetype != 1, qxface)
    qzplot = np.ma.masked_where(ifacetype != 1, qzface)
    plt.quiver(xface, zface, qxplot, qzplot, scale=scale_bndy, pivot='mid', color='green')
    qxplot = np.ma.masked_where(ifacetype != 2, qxface)
    qzplot = np.ma.masked_where(ifacetype != 2, qzface)
    plt.quiver(xface, zface, qxplot, qzplot, scale=scale_chan, pivot='mid', color='red')
    xs.plot_grid(lw = 0.1, color = 'gray')
    ax.set_yticks([])
    if j < 10: ax.set_xticks([])
    if j == 11: ax.set_xlabel('x (m)', size = 10)

In [5]:
def fig2head(fig, i, vmin, vmax, extent, scale_chan):
    
    import matplotlib.colors
    cmap1 = matplotlib.colors.LinearSegmentedColormap.from_list("", ["blue","blue","blue"])
    cmap2 = matplotlib.colors.LinearSegmentedColormap.from_list("", ["black","black","black"])
    
    j = i * 3 + 3
    ax = plt.subplot(4,3,j,aspect="equal")
    xs = flopy.plot.PlotCrossSection(model=gwf, line={"line": [(0, 0.5),(Lx, 0.5)]}, extent=extent,)
    cbct = xs.plot_array(a = icelltype, cmap = 'gray', alpha=0.4, vmin = -4, vmax = 1)
    cb = xs.contour_array(a = head,        cmap=cmap1, alpha=0.8, vmin = vmin, vmax = vmax)
    cb = xs.contour_array(a = head_analyt, cmap=cmap2, alpha=0.8, vmin = vmin, vmax = vmax)
    xs.plot_grid(lw = 0.1, color = 'black')
    ax.set_yticks([])
    if j < 10: ax.set_xticks([])
    if j == 11: ax.set_xlabel('x (m)', size = 12)
    ax.annotate("Max head error = %0.2f m" 
               %(head_diff.max()), (0.5,12.1), va='top', size = 8, bbox=dict(facecolor = 'white'))

In [6]:
def fig3():
       
    fig, axes = plt.subplots(3,2, figsize=(6.75, 5), constrained_layout=True)
   
    # FLUX MAGNITUDE
    ax = plt.subplot(321)
    ax.set_title('a) Layered Connectivity Grid\n', size = 10)
    ax.set_ylim([0.5,1.2])
    ax.set_ylabel('Flux magnitude\n(m/d)\n', size = 9)
    ax.plot(resolution_scenarios, fig3_results[:,0,0], marker = 'o', linestyle = '-') # LC - SCB
    ax.plot(resolution_scenarios, fig3_results[:,1,0], marker = 'o', linestyle = '-') # FC - SCB
    ax.plot(resolution_scenarios, np.ones(len(resolution_scenarios)), linestyle = '--', color = 'black') # Analytical
    ax.set_xticks([])
    ax.legend(['Standard', 'XT3D', 'Analytical'], loc = 'lower right', prop={'size': 8})
    
    ax = plt.subplot(322)
    ax.set_title('b) Full Connectivity Grid\n', size = 10)
    ax.set_ylim([0.5,1.2])
    ax.plot(resolution_scenarios, fig3_results[:,2,0], marker = 'o', linestyle = '-') # LC - XT3D
    ax.plot(resolution_scenarios, fig3_results[:,3,0], marker = 'o', linestyle = '-') # FC - XT3D
    ax.plot(resolution_scenarios, np.ones(len(resolution_scenarios)), linestyle = '--', color = 'black') # Analytical
    ax.set_xticks([])
    ax.set_yticks([])
    ax.legend(['Standard', 'XT3D', 'Analytical'], loc = 'lower right', prop={'size': 8})
    
    ax = plt.subplot(323)
    ax.set_ylim([-5,35])
    ax.set_ylabel('Flux angle\n(degrees)\n', size = 9)
    ax.plot(resolution_scenarios, fig3_results[:,0,1], marker = 'o', linestyle = '-') # LC - SCB
    ax.plot(resolution_scenarios, fig3_results[:,1,1], marker = 'o', linestyle = '-') # FC - SCB
    ax.plot(resolution_scenarios, 30*np.ones(len(resolution_scenarios)), linestyle = '--', color = 'black') # Analytical
    ax.set_xticks([])
    
    ax = plt.subplot(324)
    ax.set_ylim([-5,35])
    ax.plot(resolution_scenarios, fig3_results[:,2,1], marker = 'o', linestyle = '-') # LC - XT3D
    ax.plot(resolution_scenarios, fig3_results[:,3,1], marker = 'o', linestyle = '-') # FC - XT3D
    ax.plot(resolution_scenarios, 30*np.ones(len(resolution_scenarios)), linestyle = '--', color = 'black') # Analytical
    ax.set_xticks([])
    ax.set_yticks([])
    
    ax = plt.subplot(325)
    ax.set_xlabel('Grid layers within channel', size = 9)
    ax.set_ylabel('Flow error', size = 9)
    ax.plot(resolution_scenarios, fig3_results[:,0,2], marker = 'o', linestyle = '-') # LC - SCB
    ax.plot(resolution_scenarios, fig3_results[:,1,2], marker = 'o', linestyle = '-') # FC - SCB
    ax.plot(resolution_scenarios, np.zeros(len(resolution_scenarios)), linestyle = '--', color = 'black') # Analytical
    ax.set_xticks(resolution_scenarios)
    ax.set_ylim([-0.2,0.2])
    
    ax = plt.subplot(326)
    ax.set_xlabel('Grid layers within channel', size = 9)
    ax.plot(resolution_scenarios, fig3_results[:,2,2], marker = 'o', linestyle = '-') # LC - XT3D
    ax.plot(resolution_scenarios, fig3_results[:,3,2], marker = 'o', linestyle = '-') # FC - XT3D
    ax.plot(resolution_scenarios, np.zeros(len(resolution_scenarios)), linestyle = '--', color = 'black') # Analytical
    ax.set_xticks(resolution_scenarios)
    ax.set_yticks([])
    ax.set_ylim([-0.2,0.2])

    plt.tight_layout()
    plt.show()
    fig.savefig('../figures/fig3paper.png', dpi = 450)

In [7]:
def fig4():    
    
    x = dip
    
    for s in range(4): # Four scenarios
               
        fig, axs = plt.subplots(1, 2, figsize = (7,2))
        fig.suptitle = modelnames[s]
        
        # FLUX MAG
        ax = plt.subplot(121)
        for kc, kcval in enumerate(Kcontrast):
            z = qmag_results[s][kc]
            ax.plot(x, z, marker = None, linestyle = '-', label = 1/kcval)
        ax.plot(x, np.ones(len(x)), linestyle = '--', color = 'black') # Analytical
        
        if s == 3: ax.set_xlabel('Dip (degrees)', size = 9)
        ax.legend(loc = 'upper left', prop={'size': 8})
        if s < 3: ax.set_xticklabels = ([])        
        ax.set_ylim([0,3])
        ax.set_ylabel('Flux magnitude\n(m/d)', size = 8)
        ax.set_title(fig_titles[s], size = 10, loc = 'left')

        # FLUX ANGLE
        ax = plt.subplot(122)
        for kc, kcval in enumerate(Kcontrast):
            z = qang_results[s][kc]
            ax.plot(x, z, marker = None, linestyle = '-', label = 1/kcval)
        
        ax.plot(x, x, linestyle = '--', color = 'black') # Analytical
        ax.legend(loc = 'upper left', prop={'size': 8})
        ax.set_ylim([0,100]) 
        if s < 3: ax.set_xticklabels = ([])
        if s < 3: ax.set_xlabel(' ', size = 8)
        if s == 3: ax.set_xlabel('Dip (degrees)', size = 9)
        ax.set_ylabel('\nFlux direction\n(degrees)', size = 8)
        
        plt.tight_layout()
        plt.show()
        fig.savefig('../figures/fig4_%i_paper.png' %s, dpi = 450)

In [8]:
def fig5():
       
    fig, axes = plt.subplots(3,2, figsize=(6.75, 5), constrained_layout=True)
   
    # FLUX MAGNITUDE
    ax = plt.subplot(321)
    ax.set_title('a) Layered Connectivity Grid\n', size = 10)
    ax.set_ylabel('Flux magnitude\n(m/d)\n', size = 9)
    ax.plot(np.log10(anisotropic_scenarios), fig5_results[:,0,0], marker = 'o', linestyle = '-') # LC - SCB
    ax.plot(np.log10(anisotropic_scenarios), fig5_results[:,1,0], marker = 'o', linestyle = '-') # FC - SCB
    ax.plot(np.log10(anisotropic_scenarios), np.ones(len(anisotropic_scenarios)), linestyle = '--', color = 'black') # Analytical
    ax.set_xticks([])
    
    ax = plt.subplot(322)
    ax.set_title('b) Full Connectivity Grid\n', size = 10)
    ax.plot(np.log10(anisotropic_scenarios), fig5_results[:,2,0], marker = 'o', linestyle = '-') # LC - XT3D
    ax.plot(np.log10(anisotropic_scenarios), fig5_results[:,3,0], marker = 'o', linestyle = '-') # FC - XT3D
    ax.plot(np.log10(anisotropic_scenarios), np.ones(len(anisotropic_scenarios)), linestyle = '--', color = 'black') # Analytical
    ax.set_xticks([])
    ax.set_yticks([])
    
    ax = plt.subplot(323)
    ax.set_ylabel('Flux angle\n(degrees)\n', size = 9)
    ax.plot(np.log10(anisotropic_scenarios), fig5_results[:,0,1], marker = 'o', linestyle = '-') # LC - SCB
    ax.plot(np.log10(anisotropic_scenarios), fig5_results[:,1,1], marker = 'o', linestyle = '-') # FC - SCB
    ax.plot(np.log10(anisotropic_scenarios), 30*np.ones(len(anisotropic_scenarios)), linestyle = '--', color = 'black') # Analytical
    ax.set_xticks([])
    
    ax = plt.subplot(324)
    ax.plot(np.log10(anisotropic_scenarios), fig5_results[:,2,1], marker = 'o', linestyle = '-') # LC - XT3D
    ax.plot(np.log10(anisotropic_scenarios), fig5_results[:,3,1], marker = 'o', linestyle = '-') # FC - XT3D
    ax.plot(np.log10(anisotropic_scenarios), 30*np.ones(len(anisotropic_scenarios)), linestyle = '--', color = 'black') # Analytical
    ax.set_xticks([])
    ax.set_yticks([])
    
    ax = plt.subplot(325)
    ax.set_xlabel('Anisotropy ratio (log)', size = 9)
    ax.set_ylabel('Flow error', size = 9)
    ax.plot(np.log10(anisotropic_scenarios), fig5_results[:,0,2], marker = 'o', linestyle = '-') # LC - SCB
    ax.plot(np.log10(anisotropic_scenarios), fig5_results[:,1,2], marker = 'o', linestyle = '-') # FC - SCB
    ax.plot(np.log10(anisotropic_scenarios), np.zeros(len(anisotropic_scenarios)), linestyle = '--', color = 'black') # Analytical
    ax.set_xticks(np.log10(anisotropic_scenarios))
    ax.legend(['Standard', 'XT3D', 'Analytical'], loc = 'lower right', prop={'size': 8})
    
    ax = plt.subplot(326)
    ax.set_xlabel('Anisotropy ratio (log)', size = 9)
    ax.plot(np.log10(anisotropic_scenarios), fig5_results[:,2,2], marker = 'o', linestyle = '-') # LC - XT3D
    ax.plot(np.log10(anisotropic_scenarios), fig5_results[:,3,2], marker = 'o', linestyle = '-') # FC - XT3D
    ax.plot(np.log10(anisotropic_scenarios), np.zeros(len(anisotropic_scenarios)), linestyle = '--', color = 'black') # Analytical
    ax.set_xticks(np.log10(anisotropic_scenarios))
    ax.set_yticks([])
    ax.legend(['Standard', 'XT3D', 'Analytical'], loc = 'lower right', prop={'size': 8})

    plt.tight_layout()
    plt.show()
    fig.savefig('../figures/fig5paper.png', dpi = 450)

In [9]:
def table1():
    for i in range(4):
        print(modelnames[i])
        print('qmagmid = ', q_all[i][0])
        print('qangmid = ', q_all[i][1])
        print('vol_flow = ', flow_all[i], ' Analytical =', flow_analytical, '\n')